In [1]:
import pandas as pd
import time

In [2]:
data = pd.read_csv('movie_bd_v5.csv', encoding='utf-8')
data.loc[1613]['cast']

'Adam Sandler|TÃ©a Leoni|Paz Vega|Cloris Leachman|Shelbie Bruce'

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


In [149]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [121]:
answers = {}  # создадим словарь для ответов


def string_to_list(series):
    '''Распарсить строку из нескольких значений и сохранить в список'''
    series = series.str.split('|')
    return series

def get_answer_form_df(df):
    '''Напечатать название фильма в нужном формате, если он один'''
    string = df.original_title.iloc[0] + ' (' + str(df.imdb_id.iloc[0]) + ')'
    return string

def pivot_table_from_data(df,row,column,value):
    '''Собираем сводную таблицу, где на вход передаётся df = DataFrame, 
    row = индекс новой таблицы, собранный из столбца df.row
    column - столбцы новой таблицы. собранные из строк в df.column 
    value - значения, соответствующие df.value
    '''
    table = pd.DataFrame(index=list(df[row]))
    
    for index in df.index:
        for i, col in enumerate(df.loc[index][column]):
            if col in table.columns:
                table[col][df.loc[index][row]] = df.loc[index][value]
            else:
                table[col] = pd.Series(dtype=float)
                table[col][df.loc[index][row]] = df.loc[index][value]
    return table

month_dict = {
    1 : 'Январь',
    2 : 'Февраль',
    3 : 'Март',
    4 : 'Апрель',
    5 : 'Май',
    6 : 'Июнь',
    7 : 'Июль',
    8 : 'Август',
    9 : 'Сентябрь',
    10 : 'Октябрь',
    11 : 'Ноябрь',
    12 : 'Декабрь'
}

In [6]:
# преобразуем столбцу с несколькими значениями в списки

new_data = data.copy()

new_data[['cast', 'genres', 'director', 'production_companies']] = \
new_data[['cast', 'genres', 'director', 'production_companies']].apply(string_to_list)

new_data['profit'] = new_data['revenue'] - new_data['budget']

In [7]:
new_data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
829,tt1778304,5000000,205703818,Paranormal Activity 3,"[Katie Featherston, Sprague Grayden, Lauren Bi...","[Henry Joost, Ariel Schulman]",It Runs In The Family,"In 1988, evil begins to terrorize young sister...",83,[Horror],"[Paramount Pictures, Blumhouse Productions, Ro...",10/19/2011,6.0,2011,200703818
1260,tt1327773,25000000,115922175,The Butler,"[Forest Whitaker, David Banner, Michael Rainey...",[Lee Daniels],One quiet voice can ignite a revolution,A look at the life of Cecil Gaines who served ...,132,[Drama],"[Laura Ziskin Productions, Lee Daniels Enterta...",8/16/2013,7.2,2013,90922175
294,tt1144884,40000000,186167139,The Final Destination,"[Bobby Campo, Shantel VanSanten, Nick Zano, Ha...",[David R. Ellis],Rest In Pieces,After a young man's premonition of a deadly ra...,82,"[Horror, Mystery]","[New Line Cinema, Practical Pictures, Parallel...",8/26/2009,5.5,2009,146167139
1860,tt0190865,75000000,215663859,Vertical Limit,"[Chris O'Donnell, Robin Tunney, Bill Paxton, S...",[Martin Campbell],The Mountain Will Decide.,"Trapped near the summit of K2, the world's sec...",124,"[Action, Adventure, Thriller]","[Columbia Pictures Corporation, Global Enterta...",12/8/2000,6.0,2000,140663859
903,tt0278504,46000000,113714830,Insomnia,"[Al Pacino, Robin Williams, Maura Tierney, Hil...",[Christopher Nolan],A tough cop. A brilliant killer. An unspeakabl...,Two Los Angeles homicide detectives are dispat...,118,"[Crime, Mystery, Thriller]","[Section Eight, Alcon Entertainment, Witt/Thom...",5/24/2002,6.6,2002,67714830


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [122]:
answer_1 = data[data.budget == data.budget.max()]

# в словарь вставляем номер вопроса и ваш ответ на него
answers['1'] = get_answer_form_df(answer_1)

print(answers['1'])

Pirates of the Caribbean: On Stranger Tides (tt1298650)


ВАРИАНТ 2

In [10]:
# можно добавлять разные варианты решения

In [114]:
answer_1_v2 = data.loc[data.budget.idxmax()]

print('{} ({})'.format(answer_1_2['original_title'], answer_1_v2['imdb_id']))

Pirates of the Caribbean: On Stranger Tides (tt1298650)


# 2. Какой из фильмов самый длительный (в минутах)?

In [123]:
answer_2 = data[data.runtime == data.runtime.max()]

get_answer_form_df(answer_2)

'Gods and Generals (tt0279111)'

In [124]:
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [117]:
answer_3 = data[data.runtime == data.runtime.min()]

get_answer_form_df(answer_3)

'Winnie the Pooh (tt1449283)'

In [14]:
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [15]:
answer_4 = data.runtime.mean()

print(round(answer_4))

110


In [16]:
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [17]:
answer_5 = data.runtime.median()

print(answer_5)

107.0


In [18]:
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [19]:
# лучше код получения столбца profit вынести в Предобработку что в начале

In [126]:
answer_6 = new_data[new_data.profit == new_data.profit.max()]

get_answer_form_df(answer_6)

'Avatar (tt0499549)'

Второй вариант

In [127]:
answers['6'] = 'Avatar (tt0499549)'

In [22]:
display(new_data.sort_values(['profit'], ascending=False).head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit
239,tt0499549,237000000,2781505847,Avatar,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron],Enter the World of Pandora.,"In the 22nd century, a paraplegic Marine is di...",162,"[Action, Adventure, Fantasy, Science Fiction]","[Ingenious Film Partners, Twentieth Century Fo...",12/10/2009,7.1,2009,2544505847


# 7. Какой фильм самый убыточный? 

In [128]:
answer_7 = new_data[new_data.profit == new_data.profit.min()]

get_answer_form_df(answer_7)

'The Lone Ranger (tt1210819)'

In [129]:
answers['7'] = 'The Lone Ranger (tt1210819)'

Второй вариант

In [137]:
display(new_data.sort_values(['profit']).head(1))

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,ov_length,len_overview
1245,tt1210819,255000000,89289910,The Lone Ranger,"[Johnny Depp, Armie Hammer, William Fichtner, ...",[Gore Verbinski],Never Take Off the Mask,The Texas Rangers chase down a gang of outlaws...,149,"[Action, Adventure, Western]","[Walt Disney Pictures, Jerry Bruckheimer Films...",7/3/2013,6.0,2013,-165710090,7,15,386,68


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [133]:
len(new_data[new_data.revenue > new_data.budget])

1478

In [28]:
new_data.query('revenue > budget')['imdb_id'].count()

1478

In [26]:
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [134]:
answer_9 = new_data.query('release_year == 2008')\
                   .sort_values(['revenue'], ascending=False).head(1)

get_answer_form_df(answer_9)

'The Dark Knight (tt0468569)'

In [29]:
answers['9'] = 'The Dark Knight (tt0468569)'

Второй вариант

In [135]:
data_2008 = new_data[new_data.release_year == 2008]

data_2008[data_2008.revenue == data_2008.revenue.max()]

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,month,title_length,ov_length,len_overview
599,tt0468569,185000000,1001921825,The Dark Knight,"[Christian Bale, Michael Caine, Heath Ledger, ...",[Christopher Nolan],Why So Serious?,Batman raises the stakes in his war on crime. ...,152,"[Drama, Action, Crime, Thriller]","[DC Comics, Legendary Pictures, Warner Bros., ...",7/16/2008,8.1,2008,816921825,7,15,396,67


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [136]:
answer_10 = new_data.query('release_year >= 2012 & release_year <= 2014')\
                    .sort_values(['profit']).head(1)

get_answer_form_df(answer_10)


'The Lone Ranger (tt1210819)'

In [32]:
answers['10'] = 'The Lone Ranger (tt1210819)'

Второй вариант

In [139]:
data_2012_2014 = new_data[(new_data.release_year >= 2012) & (new_data.release_year <= 2014)]

data_2012_2014.loc[data_2012_2014.profit.idxmin()]['original_title']

'The Lone Ranger'

# 11. Какого жанра фильмов больше всего?

In [35]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

In [141]:
data_with_genres = new_data.explode('genres')

data_with_genres['genres'].value_counts().idxmax()

'Drama'

In [146]:
answers['11'] = 'Drama'

ВАРИАНТ 2

In [38]:
genres = {}
list_of_genres = []

for index in new_data.index:
    list_of_genres = new_data.loc[index]['genres']
    for i, genre in enumerate(list_of_genres):
        if genre in list(genres.keys()):
            genres[genre] += 1
        else:
            genres[genre] = 1

df_genres = pd.Series(data=genres, index=list(genres.keys()))

df_genres.sort_values(ascending=False).head(1)

Drama    782
dtype: int64

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [145]:
profit_movies = new_data.query('profit > 0').copy()

pivot_table_with_profit = pivot_table_from_data(profit_movies, 'imdb_id', 'genres', 'profit')

display(pivot_table_with_profit.count().sort_values(ascending=False).head(1))

Drama    560
dtype: int64

In [41]:
data_with_genres.query('profit > 0')['genres'].describe(include=object).top

'Drama'

In [147]:
answers['12'] = 'Drama'


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [42]:
pivot_table_with_directors = pivot_table_from_data(new_data, 'imdb_id', 'director', 'revenue')

pivot_table_with_directors.sum().idxmax()

'Peter Jackson'

In [43]:
data_with_directors = new_data.explode('director')

data_with_directors.groupby('director')['revenue'].sum().sort_values().tail(1)

director
Peter Jackson    6490593685
Name: revenue, dtype: int64

In [150]:
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [152]:
# разбить список жанров у каждого режиссера построчно
data_with_directors_and_genres = data_with_directors.explode('genres')

# оствляем только Action, группируем записи по режиссерам и считаем кол-во фильмов у каждого
data_with_directors_and_genres.query('genres == "Action"').groupby('director')['imdb_id'].count().idxmax()


'Robert Rodriguez'

In [45]:
# другой вариант, считаем по исходному датасету
# собираем список режиссеров (может быть больше одного), у которых есть фильмы Action
t_directors_with_action = data[data.genres.str.contains('Action')]['director'].copy()

# разбиваем списки режиссеров построчно
directors_with_action = string_to_list(t_directors_with_action).explode()

# считаем, сколько на каждого режиссера приходится упоминаний и берём максимальное
directors_with_action.value_counts().idxmax()

'Robert Rodriguez'

In [ ]:
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [46]:
# оставим фильмы только 2012 года
new_data_2012 = new_data[new_data.release_year == 2012].copy()

# разобьём списки актёров построчно, сгруппируем по актёру 
# при групировке сложим сборы по всем фильмам и найдём максимальную сумму
new_data_2012.explode('cast').groupby('cast')['revenue'].sum().idxmax()

'Chris Hemsworth'

In [47]:
data_2012 = new_data.query('release_year == 2012').copy()

pivot_table_from_data(data_2012,'imdb_id','cast','revenue').sum().idxmax()

'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [48]:
# соберем список высокобюджетных фильмов: бюджет выше среднего
high_budget_movies = new_data[new_data.budget > new_data.budget.mean()].copy()

# разберём списки актёров построчно
cast_list = high_budget_movies[['imdb_id','cast']].explode('cast')

# сгруппируем записи по актёрам и посчитаем сколько на каждого пришлось фильмов
cast_list.groupby('cast')['imdb_id'].count().idxmax()

'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [49]:
# оставим в датасете фильмы с Николасом Кейджем и вытащим стобец с жанрами
nc_genres = data[data.cast.str.contains('Nicolas Cage')]['genres']

# превратим строки в списки жанров и разобьём построчно каждый список
all_nc_genres = string_to_list(nc_genres).explode()

# считаем кол-во по каждому жанру и выбираем максимальное
all_nc_genres.value_counts().idxmax()

'Action'

In [50]:
# воспользуемся ранее собранным датафреймом

data_with_genres.explode('cast').query('cast == "Nicolas Cage"').groupby('genres')['imdb_id'].count()

genres
Action             17
Adventure           7
Animation           3
Comedy              6
Crime              10
Drama              12
Family              3
Fantasy             8
History             2
Horror              1
Mystery             3
Romance             1
Science Fiction     4
Thriller           15
War                 1
Name: imdb_id, dtype: int64

# 18. Самый убыточный фильм от Paramount Pictures

In [51]:
new_data_by_companies = new_data.explode('production_companies')

pp_movies = new_data_by_companies[new_data_by_companies.production_companies.str.contains('Paramount Pictures')]

pp_low_index = pp_movies['profit'].idxmin()

print(new_data.loc[pp_low_index]['original_title'] + ' (' + new_data.loc[pp_low_index]['imdb_id'] + ')')

K-19: The Widowmaker (tt0267626)


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [52]:
pivot_table_with_revenue = data.pivot_table(values='revenue', \
                        index=['imdb_id'],columns=['release_year'],aggfunc='sum',margins=True)

# pivot_table_with_revenue.All.sort_values(ascending=False)

pivot_table_with_revenue.loc['All'].sort_values().tail(2)

release_year
2015    2.544920e+10
All     2.934851e+11
Name: All, dtype: float64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [53]:
wb_movies = data[data.production_companies.str.contains("Warner Bros")].copy()

wb_movies['profit'] = wb_movies['revenue'] - wb_movies['budget']

pivot_with_wb_movies = wb_movies.pivot_table(values='profit', \
                        index=['imdb_id'], columns=['release_year'],aggfunc='sum',margins=True)

pivot_with_wb_movies.loc['All'].sort_values().tail(2)

release_year
2014    2.295465e+09
All     2.454360e+10
Name: All, dtype: float64

In [54]:
wb_movies.groupby('release_year')['profit'].sum().idxmax()

2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [55]:
def get_month(s):
    return time.strptime(s, '%m/%d/%Y').tm_mon

def get_season(s):
    month = time.strptime(s, '%m/%d/%Y').tm_mon
    if month in (1,2,12):
        season = 'Зима'
    elif month in (3,4,5):
        season = 'Весна'
    elif month in (6,7,8):
        season = 'Лето'
    else:
        season = 'Осень'
    return season


new_data['month'] = new_data.release_date.apply(get_month)

data_by_month = new_data.groupby(['month'])['imdb_id'].count()

month_dict[data_by_month.idxmax()]

'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [56]:
answers['22'] = new_data.query('month in (6,7,8)').imdb_id.count()

print(answers['22'])

450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [57]:
directors_by_season = new_data[['director','release_date']].explode('director')

directors_by_season['season'] = directors_by_season.release_date.apply(get_season)

display(directors_by_season.query('season == "Зима"')['director'].value_counts())

Peter Jackson            7
Steven Soderbergh        6
Clint Eastwood           6
Nancy Meyers             4
Martin Scorsese          4
                        ..
David R. Ellis           1
David Moreau             1
Sam Taylor-Johnson       1
Gary Winick              1
Christopher B. Landon    1
Name: director, Length: 358, dtype: int64

In [58]:
seasons = directors_by_season.pivot_table(values='release_date',\
                                index=['director'],columns=['season'],aggfunc='count',fill_value=0)

display(seasons.sample(5))

seasons['Зима'].idxmax()



season,Весна,Зима,Лето,Осень
director,,,,
Henry Selick,0,2,0,0
Jon Turteltaub,0,1,2,2
Jon M. Chu,0,2,1,1
Steve Antin,0,0,0,1
Jonathan Lynn,0,1,1,0


'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [59]:
new_data['title_length'] = new_data.original_title.apply(len)

display(new_data[['production_companies','title_length']].sample(3))

data_by_prod_companies = new_data.explode('production_companies')

data_by_prod_companies.groupby(['production_companies'])['title_length'].mean().idxmax()

,production_companies,title_length
313,"[StudioCanal, Dark Castle Entertainment, Don C...",8
1383,"[Canal Plus, Current Entertainment, Clubdeal, ...",9
1287,"[Reel FX Creative Studios, Relativity Media]",10


'Four By Two Productions'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [95]:
# добавляем столбец с кол-вом слов в описании фильма
new_data['len_overview'] = new_data.overview.apply(str.split).apply(len)

data_by_prod_companies = new_data.explode('production_companies')

answers['25'] = data_by_prod_companies.groupby(['production_companies'])['len_overview'].mean().idxmax()

answers['25']

'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [61]:
vote_99 = new_data.vote_average.quantile(q=0.99)

print(vote_99)

7.8


In [62]:
top_movies = new_data[new_data.vote_average > vote_99][['imdb_id','original_title','vote_average']].copy()

top_movies.sort_values('original_title')

,imdb_id,original_title,vote_average
1191,tt2024544,12 Years a Slave,7.9
128,tt2267998,Gone Girl,7.9
119,tt2015381,Guardians of the Galaxy,7.9
370,tt1375666,Inception,7.9
9,tt2096673,Inside Out,8.0
118,tt0816692,Interstellar,8.0
1800,tt0209144,Memento,7.9
34,tt3170832,Room,8.0
599,tt0468569,The Dark Knight,8.1
138,tt2278388,The Grand Budapest Hotel,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [63]:
import itertools as it


def make_couples(cast):
    """Собираем элементы из переданного списка cast в список"""
    return list(it.combinations(cast, 2))


# вытаскиваем столбец с актёрами из датасета
actors_only = new_data.cast.copy()

# собираем пары внутри каждого фильм и разбиваем построчно
actors_in_couples = actors_only.apply(make_couples).explode()

# считаем кол-во записей/фильмов для каждой пары и находим самое популярное
# на самом деле две пары подходят, но в тесте есть только один из двух вариантов
display(actors_in_couples.value_counts().head(2))

# забираем в словарь ответов первый найденный вариант
answers['27'] = actors_in_couples.value_counts().idxmax()

(Daniel Radcliffe, Rupert Grint)    8
(Daniel Radcliffe, Emma Watson)     8
Name: cast, dtype: int64

ВАРИАНТ 2 (проверка + список фильмов для выбранной пары)

In [64]:
# собираем датафрейм с актёрами и фильмами, разбиваем на пары построчно
df_cast = new_data[['cast', 'original_title']].copy()
df_cast.cast = df_cast.cast.apply(make_couples)
df_cast_in_pairs = df_cast.explode('cast')

# смотрим статику по df_cast_in_pairs
display(df_cast_in_pairs.describe(include=object)['cast'])

# список фильмов, в которых снялась наша пара
display(df_cast_in_pairs[df_cast_in_pairs.cast == answers['27']])

count                                18868
unique                               18121
top       (Daniel Radcliffe, Rupert Grint)
freq                                     8
Name: cast, dtype: object

,cast,original_title
242,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Half-Blood Prince
374,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Deathly Hallows: Part 1
497,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Philosopher's Stone
722,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Deathly Hallows: Part 2
864,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Chamber of Secrets
1312,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Goblet of Fire
1560,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Prisoner of Azkaban
1670,"(Daniel Radcliffe, Rupert Grint)",Harry Potter and the Order of the Phoenix


# Submission

In [65]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 11: 'Drama',
 '22': 450,
 '27': ('Daniel Radcliffe', 'Rupert Grint')}

In [66]:
# и убедиться что ни чего не пропустил)
len(answers)

13